In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Read in the CSV file containing layoff data
df = pd.read_csv("webDev5.csv")

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]

# Normalize the training and testing data
train_data = train_data.values.reshape(-1, 1)
test_data = test_data.values.reshape(-1, 1)
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the LSTM model on the normalized training data
X_train, y_train = train_data[:-1], train_data[1:]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# Use the LSTM model to predict the layoff numbers for the next 5 years
X_test = test_data[:-1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
predictions = []
for i in range(60):
    prediction = model.predict(X_test[-1].reshape(1, 1, 1))
    predictions.append(prediction)
    X_test = np.append(X_test, prediction.reshape(1, 1, 1), axis=0)

# Denormalize the predicted values
predictions = scaler.inverse_transform(predictions)

# Combine the historical and forecast data into a single DataFrame
dates = pd.date_range(start=df['Date'].iloc[-1], periods=len(predictions) + 1, freq='M')[1:]
forecast_df = pd.DataFrame({'Date': dates, 'Layoffs': np.concatenate([df['Layoffs'].values, predictions.reshape(-1)])})

# Plot the historical and forecast data on a graph
plt.plot(df['Date'], df['Layoffs'], label='Historical Data')
plt.plot(forecast_df['Date'], forecast_df['Layoffs'], label='Forecast')
plt.xlabel('Date')
plt.ylabel('Layoffs')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'